In [150]:
import datetime
import time
import pandas as pd
import requests
import json
from jsonpath import jsonpath
import telebot
import hmac, base64, struct, hashlib
import platform

# google验证码函数
submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Device_id':'1.0',
    'Os_type':'0',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code


def job():
    # 生成验证码
    print('生成google验证码。。。')
    google_code = get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
    data0 = {
        'username': 'Marquis',
        'password': 'qwer123456',
        'code': google_code
    }
    # 获取token
    print('获取token中。。。')
    session0 = requests.Session()
    response0  =session0.post(url=submit_url,data=data0,headers=header0)
    response0.encoding = 'utf-8'
    obj0 = json.loads(response0.text)
    token = obj0['data']['token']

    # 发送请求
    print('开始发送请求....')
    url = 'http://fundmng.bsportsadmin.com/api/manage/fund/recharge/record/list'
    header = {
        'Device_id': '1.0',
        'Os_type': '0',
        'Sign': '525f286b21988b7c6a79f623f62f9695',
        'Timestamp': '1691886184000',
        'Token': token,
        'Uid': '690',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'Version': '1.0'
    }
    session  = requests.Session()
    dic0 = {'充值日期':"createDate",'充值单号':"orderId",'用户层级':"userLevel",'账户名':"username",'姓名':"reallyName",'VIP等级':"vipLevel",'充值金额':"amount",'到账金额':"payAmount"}
    dic = {'充值日期':[],'充值单号':[],'用户层级':[],'账户名':[],'姓名':[],'VIP等级':[],'充值金额':[],'到账金额':[]}
    for page in range(1,100):
        print(f'第{page}页。。。')
        data = {
            'dateType':'1',
            'startTime':1693929600000,
            'endTime':1694015999999,
            'userType':'-1',
            'orderStatus':'0,1,2,3,4,5,6,7,8,9',
            'agentType':'-1',
            'page':page,
            'size':500,
            'paymentIdList':'1,2,3,4,5,6,7,8,9,10,11,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,2000,2001,2002,2003,2004,2005,2006,3000,3001,4000,4001,4002,4003,4004,8000,8001,8002,10000,10001',
            'coinCode':'CNY'
        }

        response = session.post(url=url,headers=header,data=data)
        response.encoding = 'utf8'

        # 提取数据

        obj = json.loads(response.text)
        for i in obj['data']['list']:
            if (i["userLevel"] == '多次拉单不付款层级' and i["payAmount"] ==0 ):
                # if (i["userLevel"] == '多次拉单不付款层级' and i["payAmount"] ==0 and i["vipLevel"] >0):
                for k in dic:
                    dic[k].append(i[dic0[k]])
    # 生成结果数据
    df = pd.DataFrame(dic)
    df.insert(1,'时间',df['充值日期'].map(lambda x:time.strftime("%H:%M")))
    df['充值日期']= df['充值日期'].map(lambda x:time.strftime("%Y-%m-%d"))


    result = df.groupby('账户名').agg({'账户名':len}).rename(columns={'账户名':'数量'})
    result.reset_index(inplace=True)
    print(result.sort_values('数量',ascending=False))
    return df
df = job()

生成google验证码。。。
获取token中。。。
开始发送请求....
第1页。。。
第2页。。。
第3页。。。
第4页。。。
第5页。。。
第6页。。。
第7页。。。
第8页。。。
第9页。。。
第10页。。。
第11页。。。
第12页。。。
第13页。。。
第14页。。。
第15页。。。
第16页。。。
第17页。。。
第18页。。。
第19页。。。
第20页。。。
第21页。。。
第22页。。。
第23页。。。
第24页。。。
第25页。。。
第26页。。。
第27页。。。
第28页。。。
第29页。。。
第30页。。。
第31页。。。
第32页。。。
第33页。。。
第34页。。。
第35页。。。
第36页。。。
第37页。。。
第38页。。。
第39页。。。
第40页。。。
第41页。。。
第42页。。。
第43页。。。
第44页。。。
第45页。。。
第46页。。。
第47页。。。
第48页。。。
第49页。。。
第50页。。。
第51页。。。
第52页。。。
第53页。。。
第54页。。。
第55页。。。
第56页。。。
第57页。。。
第58页。。。
第59页。。。
第60页。。。
第61页。。。
第62页。。。
第63页。。。
第64页。。。
第65页。。。
第66页。。。
第67页。。。
第68页。。。
第69页。。。
第70页。。。
第71页。。。
第72页。。。
第73页。。。
第74页。。。
第75页。。。
第76页。。。
第77页。。。
第78页。。。
第79页。。。
第80页。。。
第81页。。。
第82页。。。
第83页。。。
第84页。。。
第85页。。。
第86页。。。
第87页。。。
第88页。。。
第89页。。。
第90页。。。
第91页。。。
第92页。。。
第93页。。。
第94页。。。
第95页。。。
第96页。。。
第97页。。。
第98页。。。
第99页。。。
             账户名  数量
29  fxgjbgr64345  21
54    q546418457  20
13  aa2290030902  14
22      chao4429  12
81     zxz334964  11
..           ...  ..
16     ag5678886   1
18      an1

In [174]:
import numpy as np
result = df.groupby('账户名').agg({'账户名':len,'充值金额':np.mean}).rename(columns={'账户名':'数量','充值金额':'平均充值金额'})
result.reset_index(inplace=True)
result

,账户名,数量,平均充值金额
0,0660lin,2,750.500000
1,402app,10,100.500000
2,6686688king8,2,150.000000
3,714037733Y,5,10460.000000
4,8765923ww,4,300.000000
...,...,...,...
77,zcx666,1,100.000000
78,zcy19971018,2,1250.000000
79,zd253095551,2,2900.000000
80,zq130304,3,2666.666667


In [168]:
result.loc[result['账户名']=='0660lin','数量'].values[0]

2

In [175]:
df
result2 = result.loc[result['数量']>10,:]

,充值日期,时间,充值单号,用户层级,账户名,姓名,VIP等级,充值金额,到账金额
0,2023-09-07,17:27,CZ2023090623599ZYNHW,多次拉单不付款层级,lxj24960350,李晓君,0,2000.0,0
1,2023-09-07,17:27,CZ2023090623559ZXTTP,多次拉单不付款层级,rnmd330,潘韦刚,0,100.0,0
2,2023-09-07,17:27,CZ2023090623329ZR7OD,多次拉单不付款层级,shi882023,陈东浩,0,500.0,0
3,2023-09-07,17:27,CZ2023090623139ZLIMQ,多次拉单不付款层级,0660lin,林燕峰,0,1001.0,0
4,2023-09-07,17:27,CZ2023090623059ZIUII,多次拉单不付款层级,gaoguang3,高福光,0,1001.0,0
...,...,...,...,...,...,...,...,...,...
385,2023-09-07,17:27,CZ2023090600129RFN0P,多次拉单不付款层级,hpy123456788,韩鹏远,0,100.0,0
386,2023-09-07,17:27,CZ2023090600119RFFAN,多次拉单不付款层级,q546418457,张奇,0,400.0,0
387,2023-09-07,17:27,CZ2023090600109RF413,多次拉单不付款层级,mnb167856,胡世阳,0,3000.0,0
388,2023-09-07,17:27,CZ2023090600099REPXM,多次拉单不付款层级,mnb167856,胡世阳,0,3000.0,0


In [184]:
len(result2.index)

5

In [183]:
fp = open('result.txt','w')
for name in result2.index:
    fp.write(f'会员等级：{df2.loc[name,"VIP等级"]}\n')
    fp.write(f'账号：{name}\n')
    fp.write(f'次平均金额：{result2.loc[name,"平均充值金额"].mean()}\n')
    fp.write(f'24小时申请次数：{result2.loc[name,"数量"]}\n')
    fp.write('---------------------------------\n')
fp.write(f'总计：{len(result2.indwx)}')
fp.close()

287

16

12

12

34

...

13

24

12

34

AttributeError: 'DataFrame' object has no attribute 'indwx'

In [161]:
df2 = df.copy()
df2.set_index('账户名',inplace=True)
df2.head()

,充值日期,时间,充值单号,用户层级,姓名,VIP等级,充值金额,到账金额
账户名,,,,,,,,
lxj24960350,2023-09-07,17:27,CZ2023090623599ZYNHW,多次拉单不付款层级,李晓君,0,2000.0,0
rnmd330,2023-09-07,17:27,CZ2023090623559ZXTTP,多次拉单不付款层级,潘韦刚,0,100.0,0
shi882023,2023-09-07,17:27,CZ2023090623329ZR7OD,多次拉单不付款层级,陈东浩,0,500.0,0
0660lin,2023-09-07,17:27,CZ2023090623139ZLIMQ,多次拉单不付款层级,林燕峰,0,1001.0,0
gaoguang3,2023-09-07,17:27,CZ2023090623059ZIUII,多次拉单不付款层级,高福光,0,1001.0,0


In [172]:
import os
os.getcwd()

'C:\\Users\\User\\IdeaProjects\\project1\\Bsport\\充值拉单预警\\0907'

In [16]:
import datetime
import time
import pandas as pd
import requests
import json
from jsonpath import jsonpath
import telebot
import hmac, base64, struct, hashlib
import platform
from selenium import webdriver

submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Device_id':'1.0',
    'Os_type':'0',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code
# 生成验证码
google_code = get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': google_code
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']
token

'5c0503dc70a6463b9e398eade4db1b60'

In [142]:
url = 'http://fundmng.bsportsadmin.com/api/manage/fund/recharge/record/list'
data = {
    'dateType':'1',
    'startTime':1693929600000,
    'endTime':1694015999999,
    'userType':'-1',
    'orderStatus':'0,1,2,3,4,5,6,7,8,9',
    'agentType':'-1',
    'page':1,
    'size':500,
    'paymentIdList':'1,2,3,4,5,6,7,8,9,10,11,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,2000,2001,2002,2003,2004,2005,2006,3000,3001,4000,4001,4002,4003,4004,8000,8001,8002,10000,10001',
    'coinCode':'-1'
}

header = {
    'Device_id': '1.0',
    'Os_type': '0',
    'Sign': '525f286b21988b7c6a79f623f62f9695',
    'Timestamp': '1691886184000',
    'Token': '0f6bb5c38ff047f9abbd32464931365d',
    'Uid': '690',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Version': '1.0'
}
response = requests.post(url=url,headers=header,data=data)
response.encoding = 'utf8'
response.text

'{"code":1,"data":{"endRow":500,"glRechargeAllCollect":null,"hasNextPage":true,"hasPreviousPage":false,"isFirstPage":true,"isLastPage":false,"list":[{"agentType":0,"amount":200000.00000000,"appType":0,"attachments":"","bankId":-1,"bankName":"TCB","blockAddress":"","cardNo":"1903******3016","cardUsername":"LE DUY BAO","channelId":454,"channelName":"KolaPay","clientType":1,"coin":"VND","createDate":1694015998000,"deviceId":"","fee":0E-8,"first":0,"hasHasten":false,"ip":"171.255.66.104","keyword":"","lastUpdate":1694016064000,"limitType":0,"matchWithdrawOrderId":"","matchWithdrawUsername":"","merchantCode":"10076","merchantId":40,"merchantName":"KolaPay","orderId":"CZ2023090623599ZYRTP","orgPaymentId":2004,"originalOrderId":"","payAddress":"","payAmount":200000.00000000,"payRate":0,"payTime":1694016064000,"paymentDate":null,"paymentId":2004,"paymentName":"","protocol":"","rate":0,"reallyName":"Quach manh hai","remark":"","showType":0,"status":1,"subStatus":1,"sucApvAmount":0,"sucApvOperat

In [144]:
dic0 = {'充值日期':"createDate",'充值单号':"orderId",'用户层级':"userLevel",'账户名':"username",'姓名':"reallyName",'VIP等级':"vipLevel",'充值金额':"amount",'到账金额':"payAmount"}
dic = {'充值日期':[],'充值单号':[],'用户层级':[],'账户名':[],'姓名':[],'VIP等级':[],'充值金额':[],'到账金额':[]}
obj = json.loads(response.text)
for i in obj['data']['list']:
    # if (i["userLevel"] == '多次拉单不付款层级' and i["payAmount"] ==0 ):
    if (i["userLevel"] == '多次拉单不付款层级' ):
        for k in dic:
            dic[k].append(i[dic0[k]])

df = pd.DataFrame(dic)
df.insert(1,'时间',df['充值日期'].map(lambda x:time.strftime("%H:%M")))
df['充值日期']= df['充值日期'].map(lambda x:time.strftime("%Y-%m-%d"))
df.shape

(6, 9)

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   充值日期    6 non-null      object 
 1   时间      6 non-null      object 
 2   充值单号    6 non-null      object 
 3   用户层级    6 non-null      object 
 4   账户名     6 non-null      object 
 5   姓名      6 non-null      object 
 6   VIP等级   6 non-null      int64  
 7   充值金额    6 non-null      float64
 8   到账金额    6 non-null      float64
dtypes: float64(2), int64(1), object(6)
memory usage: 560.0+ bytes


In [87]:
dic0 = {'充值日期':"createDate",'充值单号':"orderId",'用户层级':"userLevel",'账户名':"username",'姓名':"reallyName",'充值金额':"amount",'到账金额':"payAmount"}
dic = {'充值日期':[],'充值单号':[],'用户层级':[],'账户名':[],'姓名':[],'充值金额':[],'到账金额':[]}
# jsonpath(json.loads(response.text),'$..orderId')
for k in dic:
    dic[k].extend(jsonpath(json.loads(response.text),f'$..{dic0[k]}'))
dic

{'充值日期': [1694015998000,
  1694015998000,
  1694015997000,
  1694015995000,
  1694015994000,
  1694015992000,
  1694015991000,
  1694015990000,
  1694015989000,
  1694015983000,
  1694015982000,
  1694015979000,
  1694015975000,
  1694015975000,
  1694015975000,
  1694015974000,
  1694015973000,
  1694015971000,
  1694015968000,
  1694015966000,
  1694015962000,
  1694015962000,
  1694015961000,
  1694015958000,
  1694015946000,
  1694015944000,
  1694015942000,
  1694015939000,
  1694015938000,
  1694015936000,
  1694015928000,
  1694015926000,
  1694015924000,
  1694015922000,
  1694015920000,
  1694015918000,
  1694015918000,
  1694015916000,
  1694015916000,
  1694015916000,
  1694015914000,
  1694015913000,
  1694015911000,
  1694015909000,
  1694015902000,
  1694015900000,
  1694015898000,
  1694015897000,
  1694015896000,
  1694015890000,
  1694015890000,
  1694015885000,
  1694015885000,
  1694015883000,
  1694015882000,
  1694015880000,
  1694015879000,
  1694015878000,
  1694

In [88]:
import pandas as pd
df = pd.DataFrame(dic)
df.insert(1,'时间',df['充值日期'].map(lambda x:time.strftime("%H:%M")))
df['充值日期']= df['充值日期'].map(lambda x:time.strftime("%Y-%m-%d"))
df

,充值日期,时间,充值单号,用户层级,账户名,姓名,充值金额,到账金额
0,2023-09-07,15:16,CZ2023090623599ZYRTP,初审后层级2.1,Swit86,Quach manh hai,200000.0,200000.0
1,2023-09-07,15:16,CZ2023090623599ZYRST,大客会员层级,x337099712,杨鑫,500.0,500.0
2,2023-09-07,15:16,CZ2023090623599ZYRSB,正常层级,Truongan123,VO TRUONG AN,7000000.0,7000000.0
3,2023-09-07,15:16,CZ2023090623599ZYRQZ,seo初审后层级2.1,liu15890229510,刘奇,200.0,200.0
4,2023-09-07,15:16,CZ2023090623599ZYRQ0,初审后层级2.1,Thaoquyen123,CU THAI MINH,300000.0,300000.0
...,...,...,...,...,...,...,...,...
495,2023-09-07,15:16,CZ2023090623449ZUPIP,初审后层级2.1,sonama21,PHAN TUNG SON,200000.0,0.0
496,2023-09-07,15:16,CZ2023090623449ZUPHX,初审后层级2.1,danganhtuan14791,DANG ANH TUAN,500000.0,500000.0
497,2023-09-07,15:16,CZ2023090623449ZUPHQ,初审后层级2.1,bilubilubilu110,黄开耀,110.0,110.0
498,2023-09-07,15:16,CZ2023090623449ZUPGS,初审后层级2,a271871104,徐曾麟,100.0,0.0


In [76]:
df2.group

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   充值时间    2500 non-null   datetime64[ns]
 1   充值单号    2500 non-null   object        
 2   用户层级    2500 non-null   object        
 3   账户名     2500 non-null   object        
 4   姓名      2500 non-null   object        
 5   充值金额    2500 non-null   float64       
 6   到账金额    2500 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 136.8+ KB


In [117]:
df2.groupby('账户名').agg({'账户名':len}).rename(columns={'账户名':'数量'})

,数量
账户名,
0660lin,10
10zxcvbnm,5
123asdplm,12
402app,10
6686688king8,2
...,...
zcx666,1
zcy19971018,15
zd253095551,2


In [133]:
df3.sort_values('数量',ascending=False)

,账户名,数量
118,q10368527,52
54,ZX200122,39
82,fxgjbgr64345,23
14,Cuong1993789,22
33,Nguyenvui1986,20
...,...,...
132,testcesardlcs123,1
135,thomluntaongo39,1
136,toan19882,1
7,A18575969297,1


In [129]:
df2.loc[df2['账户名']=='q10368527',]

,充值时间,充值单号,关联单号,账户类型,用户层级,账户名,姓名,付款人姓名,币种,充款金额,...,银行卡号,订单汇率,实际汇率,提单USDT,实际支付USDT,交易哈希值,匹配提现单号,提现会员账号,钱包地址,原始充值方式
154,2023/9/7 4:23,CZ202309070423A1JON3,NaN,会员,多次拉单不付款层级,q10368527,邵春宇,无,CNY,200,...,\t,0.0,0.0,0.0,0.0,NaN,\t,\t,\t,极速支付宝
162,2023/9/7 3:35,CZ202309070335A1D1GP,NaN,会员,多次拉单不付款层级,q10368527,邵春宇,无,CNY,200,...,\t,0.0,0.0,0.0,0.0,NaN,\t,\t,\t,极速支付宝
163,2023/9/7 3:34,CZ202309070334A1CTES,NaN,会员,多次拉单不付款层级,q10368527,邵春宇,无,CNY,100,...,\t,0.0,0.0,0.0,0.0,NaN,\t,\t,\t,极速支付宝
164,2023/9/7 3:31,CZ202309070331A1CJQF,NaN,会员,多次拉单不付款层级,q10368527,邵春宇,无,CNY,200,...,\t,0.0,0.0,0.0,0.0,NaN,\t,\t,\t,极速支付宝
165,2023/9/7 3:29,CZ202309070329A1C2L7,NaN,会员,多次拉单不付款层级,q10368527,邵春宇,无,CNY,300,...,\t,0.0,0.0,0.0,0.0,NaN,\t,\t,\t,网银转账
166,2023/9/7 3:27,CZ202309070327A1BWUU,NaN,会员,多次拉单不付款层级,q10368527,邵春宇,无,CNY,300,...,\t,0.0,0.0,0.0,0.0,NaN,\t,\t,\t,支付宝H5
167,2023/9/7 3:27,CZ202309070327A1BW92,NaN,会员,多次拉单不付款层级,q10368527,邵春宇,无,CNY,200,...,\t,0.0,0.0,0.0,0.0,NaN,\t,\t,\t,支付宝支付
168,2023/9/7 3:26,CZ202309070326A1BQGV,NaN,会员,多次拉单不付款层级,q10368527,邵春宇,无,CNY,200,...,\t,0.0,0.0,0.0,0.0,NaN,\t,\t,\t,极速支付宝
169,2023/9/7 3:25,CZ202309070325A1BM9S,NaN,会员,多次拉单不付款层级,q10368527,邵春宇,无,CNY,200,...,\t,0.0,0.0,0.0,0.0,NaN,\t,\t,\t,极速支付宝
170,2023/9/7 3:24,CZ202309070324A1BBTT,NaN,会员,多次拉单不付款层级,q10368527,邵春宇,无,CNY,200,...,\t,0.0,0.0,0.0,0.0,NaN,\t,\t,\t,网银转账


In [17]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"